In [55]:
!pip install -q pyomo

In [56]:
from pyomo.environ import *
import numpy as np

Question 1:

min 2x1 − 2x2 + 2x3

s.t. x1 ≥ 2, x2 ≥ 1, x3 ≥ 1,

2x1 − x2 + 2x3 ≤ 6,

x1 − 4x2 + 2x3 ≤ −15,

−2x1 + 2x2 + x3 ≤ −8.

In [57]:
model = ConcreteModel()

In [58]:
M = 3 # variables
N = 3 # constraints

In [59]:
ub = np.array([np.inf]*3)
lb = np.array([2,1,1])
objec_coef = np.array([2,-2,2])
constr_coef = np.array([[2,-1,2],[1,-4,2],[-2,2,1]])
constr_rhs = np.array([6,-15,-8])
row_indices = np.arange(N)
col_indices = np.arange(M)
model.x = Var(col_indices)
model.constraints = ConstraintList()
for i in col_indices:
  model.x[i].setlb(lb[i])
  model.x[i].setub(ub[i])
for i in row_indices:
  model.constraints.add(sum(constr_coef[i][j]*model.x[j] for j in col_indices)<=constr_rhs[i])
model.objective = Objective(expr = summation(objec_coef,model.x))#by default sense is minimize
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [60]:
!apt-get install -y -qq glpk-utils

In [61]:
opt = SolverFactory('glpk',executable = '/usr/bin/glpsol')
result  = opt.solve(model)
print(result)

print("Solver Status",result.solver.status)
print('Solver Termination conditon',result.solver.Termination_condition)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01503610610961914

Solver Status ok
Solver Termination conditon other


<b>Question 1  Solution:</b>

using glpk the output is ::

* Solver Status ok
* Solver Termination conditon other


By using the glpk solver we got the termination condition as other,lower bound is -inf and upper bound is +inf so there is no clarity in the solution like we can not infer correct reason for termination

So we are trying cbc solver

In [62]:
!apt-get install -y -qq coinor-cbc

In [63]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model)
print(result)
print('Solver Status: ',result.solver.status)
print('Solver termination condition',result.solver.termination_condition)
print('Solver Time ',result.solver.time)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.023841381072998047

Solver Status:  warning
Solver termination condition infeasible
Solver Time  0.023841381072998047


<b>Q 1 Solution Continuation ...</b>

using cbc the output is ::

* Solver Status:  warning :- we got warning as there is no solution
* Solver termination condition infeasible

As we tried solving using glpk and we don't got enough information about termination Condition . Now as we have used cbc solver we got information as termination condition is infeasible . 

So Sometimes glpk solver fails to give information that is why  we use cbc solver. Now using cbc solver we got exact reason for termination


Are they meaningful? No for glpk and Yes for cbc

* for glpk termination condition is other and upper bound is inf and lower bound is -inf so it is not at all meaningful

* for cbc termination condition is infeasible which says there is no solution for given lpp with the given constraitns, Yes it is meaningful


***Question 2***

In order to address the issue faced during solving problem (OP), you decide to use only
cbc solver and intend to perform the following resolution steps independent of each other
and check the result of the solver in each case.


***2(A)***

* Change the optimization problem to a maximization problem and re-solve.

In [64]:
#2(a)
model.objective.set_sense(maximize) #changing the objective function into maximize
model.pprint()#by using the pprint() we can check whether the sense is updated or not

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :   6.

In [65]:
result = opt_cbc.solve(model)
print(result)
print('Solver Status: ',result.solver.status)
print('Solver termination condition',result.solver.termination_condition)
print('Solver Time ',result.solver.time)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.02454209327697754

Solver Status:  warning
Solver termination condition infeasible
Solver Time  0.02454209327697754


***2(A) Solution***

So we changed the sense to maximize and solved using cbc constraint still we got infeasible solution

the output got through cbc solver is:



*   Solver Status:  warning
*   Solver termination condition infeasible






***2(B)***

Add upper bounds on x2, x3 as x2 ≤ 5, x3 ≤ 10 and re-solve.




In [66]:
#2b
model.objective.set_sense(minimize) #first we need to change the sense because we changed it int the 2(a) to revert the problem
model.x[1].setub(5)#giving upper bound of X2 as 5
model.x[2].setub(10)#giving upper bound of X3 as 10
result = opt_cbc.solve(model)
print(result)
print('Solver Status: ',result.solver.status)
print('Solver termination condition',result.solver.termination_condition)
print('Solver Time ',result.solver.time)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.02149176597595215

Solver Status:  warning
Solver termination condition infeasible
Solver Time  0.02149176597595215


***2(B) Solution***

So we added upper bounds to X2 and X3 to the original problem 

and used the cbc solver to solve and we got the solution as infeasible

This is output of cbc solver ::
* Solver Status:  warning
* Solver termination condition infeasible



***2(c) Question***

Add a new constraint −x2 + x3 ≤ 16 and re-solve.


In [67]:
#2c
model.x[1].setub(np.inf)#resetting the upper bound to revert to the original problem
model.x[2].setub(np.inf)#resetting the upper bound to revert to the original problem
new_constr_coef = [0,-1,1]
new_constr_rhs = 16
model.constraints.add(expr = sum(new_constr_coef[i]*model.x[i] for i in col_indices)<=16)# adding new constraint 
model.pprint()#using pprint() to check the constraint is added or not

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=4, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :  

In [68]:
result = opt_cbc.solve(model)
print(result)
print('Solver Status: ',result.solver.status)
print('Solver termination condition',result.solver.termination_condition)
print('Solver Time ',result.solver.time)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: 11.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 0
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.03184366226196289

Solver Status:  warning
Solver termination condition infeasible
Solver Time  0.03184366226196289


***2(c) Solution***

So we added the new constraint to the original lpp and solved using cbc solver and we got the answer as infeasible solution

This is the output of cbc solver 
* Solver Status:  warning
* Solver termination condition infeasible


***2(d) Question***

Deactivate the objective of problem (OP) and construct a new objective of the form
min x1 +2x2 +3x3 

and deactivate the constraint 2x1 −x2 +2x3 ≤ 6 

and add a different
constraint x2 + x3 ≥ 25.


In [69]:
#2d
model.constraints[4].deactivate()#deactivating the constraint added in 2(c) to revert to the original lpp
model.pprint()#checking using pprint() whether constraint is deactivated or not

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints : Size=4, Index=constraints_index, Active=True
        Key : Lower : Body                    : Upper : Active
          1 :  -Inf :  2*x[0] - x[1] + 2*x[2] :  

In [70]:
model.constraints[1].deactivate() #deactivating the constraint 1
model.objective.deactivate()# deactivating the original objective
new_obj_coef = np.array([1,2,3])
model.new_objective = Objective(expr = summation(new_obj_coef,model.x))#defining the new objective to the original lpp,by default sense is minimize
new_constr_coeff = np.array([0,1,1])
new_constr_rhs = 25
model.constraints.add(expr = summation(new_constr_coeff,model.x)>=new_constr_rhs)#adding new constraint as suggested in 2(d)
model.pprint()#checking whether all the changes are made using pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     2 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     1 :  None :  None : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + 2*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : 2*x[0] - 2*x[1] + 2*x[2]

1 Constraint Declarations
    constraints

In [71]:
result = opt_cbc.solve(model)
print(result)
print('Solver Status: ',result.solver.status)
print('Solver termination condition',result.solver.termination_condition)
print('Solver Time ',result.solver.time)


Problem: 
- Name: unknown
  Lower bound: 79.5
  Upper bound: 79.5
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.034865379333496094
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver Status:  ok
Solver termination condition optimal
Solver Time  0.034865379333496094


***2(d) Solution***

we have done the given modifications as per the question and used cbc solver 

the output is optimum solution 


In [72]:
#display solution
print( '\n Objective value =',model.new_objective())

print('\n Decision variables')
for i in col_indices:
  print('X[',i,']=',model.x[i].value)
print('\n Constraints')
model.constraints.display()


 Objective value = 79.5

 Decision variables
X[ 0 ]= 28.5
X[ 1 ]= 24.0
X[ 2 ]= 1.0

 Constraints
constraints : Size=5
    Key : Lower : Body  : Upper
      2 :  None : -65.5 : -15.0
      3 :  None :  -8.0 :  -8.0
      5 :  25.0 :  25.0 :  None


***2(d) solution continuation***

Since we got the optimal solution the values of decision variables are



1.   $X_1$ = 28.5
2.   $X_2$ = 24
3.   $X_3$ = 1


Objective optimum value is 79.5

***Active and Inactive Status:***

* Constraint1 : LHS != RHS  that implies inactive
* Constraint2 : LHS  = RHS  that implies active
* Constraint3 : LHS  = RHS  that implies active 
